<a href="https://colab.research.google.com/github/simeonubi/Disaster_classifier-/blob/main/Disaster_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes
import xgboost as xgb
from sklearn import svm
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from gensim.models import word2vec
import pickle
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import cleaned data
df = pd.read_pickle('/content/drive/MyDrive/data 2/clean_data.pkl')

In [ ]:
df.head()

,text,target
0,witnessed first demolition derby today psa exc...,0
1,help love suffer selfesteem wounds today,0
2,greer man dies wreck,1
3,harbhajansingh believeis stuart broad destroye...,0
4,alfapedia might burst bomb making suicide bomber,1


In [ ]:
#split data into train and validation
#train, val = train_test_split(df, test_size= 0.20)

In [ ]:
#appends each sentence into a list for train
#list_train=[]
#for sentance in train.text:
    #list_train.append(sentance.split())

In [ ]:
#appends each sentence into a list for validation
#list_val=[]
#for sentance in val.text:
    #list_val.append(sentance.split())

In [ ]:
#convert to list to array
#list_train = np.array(list_train)
#list_val = np.array(list_val)

In [ ]:
#val.shape

(1523, 2)

In [ ]:
#get the target variable
target = df.target

In [ ]:
#get word embedding file
EMBEDDING_FILE = '/content/drive/My Drive/Word2Vec/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
i=0
list_of_sentance=[]
for sentance in df.text:
    list_of_sentance.append(sentance.split())

In [ ]:
#loads word embedding
w2v = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#word embedding vector for the word "test"
w2v.word_vec("test")[:10]

array([-0.14257812, -0.03686523,  0.13574219, -0.06201172,  0.07958984,
        0.01904297, -0.08154297, -0.12792969, -0.02954102,  0.23632812],
      dtype=float32)

In [ ]:
#gets words in word embadding that occurs more than 5 time
w2v_words = list(w2v.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  3000000
sample words  ['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are', 'I', 'have', 'he', 'will', 'has', '####', 'his', 'an', 'this', 'or', 'their', 'who', 'they', 'but', '$', 'had', 'year', 'were', 'we', 'more', '###', 'up', 'been', 'you', 'its', 'one', 'about', 'would', 'which', 'out']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


## **Average Word2Vec**

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(300) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

  0%|          | 0/7613 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
100%|██████████| 7613/7613 [13:54<00:00,  9.12it/s]

7613
300


In [ ]:
numpy_vec = np.array(sent_vectors)

In [ ]:
#split the word vectors into train and validation set
X_train_w2v, y_train_w2v  = sent_vectors[0:6193], target[0:6193]
X_val_w2v, y_val_w2v = sent_vectors[6193:], target[6193:]

## **SVM**

In [ ]:
#support vector machine algorithm
svm_w2v = svm.SVC()
svm_w2v.fit(X_train_w2v, y_train_w2v)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#prediction and evaluation on train data with word2vec
pred_train = svm_w2v.predict(X_train_w2v) # Training Evaluation
print(classification_report(y_train_w2v, pred_train))
print()
print('Confusion Matrix: \n', confusion_matrix(y_train_w2v, pred_train))
print()
print('Accuracy :', accuracy_score(y_train_w2v, pred_train))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      3514
           1       0.92      0.80      0.86      2679

    accuracy                           0.88      6193
   macro avg       0.89      0.87      0.88      6193
weighted avg       0.89      0.88      0.88      6193


Confusion Matrix: 
 [[3320  194]
 [ 530 2149]]

Accuracy : 0.8830938155982561


In [ ]:
#prediction and evaluation on test data with word2vec
pred_val = svm_w2v.predict(X_val_w2v) # Test Evaluation
print(classification_report(y_val_w2v, pred_val))
print()
print('Confusion Matrix: \n', confusion_matrix(y_val_w2v, pred_val))
print()
print('Accuracy :', accuracy_score(y_val_w2v, pred_val))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       828
           1       0.83      0.70      0.76       592

    accuracy                           0.81      1420
   macro avg       0.82      0.80      0.80      1420
weighted avg       0.81      0.81      0.81      1420


Confusion Matrix: 
 [[742  86]
 [179 413]]

Accuracy : 0.8133802816901409


## **Decision** **Tree**

In [ ]:
#Decision tree algorithm with word2vec
decision_w2v = DecisionTreeClassifier()
decision_w2v.fit(X_train_w2v, y_train_w2v)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
#prediction and evaluation on train data with word2vec
pred_train = decision_w2v.predict(X_train_w2v) # Training Evaluation
print(classification_report(y_train_w2v, pred_train))
print()
print('Confusion Matrix: \n', confusion_matrix(y_train_w2v, pred_train))
print()
print('Accuracy :', accuracy_score(y_train_w2v, pred_train))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3514
           1       1.00      0.96      0.98      2679

    accuracy                           0.98      6193
   macro avg       0.98      0.98      0.98      6193
weighted avg       0.98      0.98      0.98      6193


Confusion Matrix: 
 [[3506    8]
 [ 100 2579]]

Accuracy : 0.9825609559179719


In [ ]:
#prediction and evaluation on test data with word2vec
pred_val = decision_w2v.predict(X_val_w2v) # Test Evaluation
print(classification_report(y_val_w2v, pred_val))
print()
print('Confusion Matrix: \n', confusion_matrix(y_val_w2v, pred_val))
print()
print('Accuracy :', accuracy_score(y_val_w2v, pred_val))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       828
           1       0.61      0.63      0.62       592

    accuracy                           0.68      1420
   macro avg       0.67      0.67      0.67      1420
weighted avg       0.68      0.68      0.68      1420


Confusion Matrix: 
 [[591 237]
 [219 373]]

Accuracy : 0.6788732394366197


## **Random Forest Classifier**

In [ ]:
#random Forest Classifier algorithm with word2vec
rand_w2v = RandomForestClassifier()
rand_w2v.fit(X_train_w2v, y_train_w2v)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#prediction and evaluation on train data with word2vec
pred_train = rand_w2v.predict(X_train_w2v) # Training Evaluation
print(classification_report(y_train_w2v, pred_train))
print()
print('Confusion Matrix: \n', confusion_matrix(y_train_w2v, pred_train))
print()
print('Accuracy :', accuracy_score(y_train_w2v, pred_train))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3514
           1       0.99      0.97      0.98      2679

    accuracy                           0.98      6193
   macro avg       0.98      0.98      0.98      6193
weighted avg       0.98      0.98      0.98      6193


Confusion Matrix: 
 [[3491   23]
 [  85 2594]]

Accuracy : 0.9825609559179719


In [ ]:
#prediction and evaluation on test data with word2vec
pred_val = rand_w2v.predict(X_val_w2v) # Test Evaluation
print(classification_report(y_val_w2v, pred_val))
print()
print('Confusion Matrix: \n', confusion_matrix(y_val_w2v, pred_val))
print()
print('Accuracy :', accuracy_score(y_val_w2v, pred_val))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       828
           1       0.81      0.64      0.72       592

    accuracy                           0.79      1420
   macro avg       0.79      0.77      0.77      1420
weighted avg       0.79      0.79      0.78      1420


Confusion Matrix: 
 [[740  88]
 [213 379]]

Accuracy : 0.7880281690140845


## **Boosting Algorithm**

In [ ]:
X_train_w2vx, y_train_w2vx  = numpy_vec[0:6193], target[0:6193]
X_val_w2vx, y_val_w2vx = numpy_vec[6193:], target[6193:]

In [ ]:
#boosting algorithm with word2vec
xgb_w2v = xgb.XGBClassifier()
xgb_w2v.fit(X_train_w2vx, y_train_w2vx)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
#prediction and evaluation on train data with word2vec
pred_train = xgb_w2v.predict(X_train_w2vx) # Training Evaluation
print(classification_report(y_train_w2vx, pred_train))
print()
print('Confusion Matrix: \n', confusion_matrix(y_train_w2vx, pred_train))
print()
print('Accuracy :', accuracy_score(y_train_w2vx, pred_train))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89      3514
           1       0.88      0.79      0.84      2679

    accuracy                           0.86      6193
   macro avg       0.87      0.86      0.86      6193
weighted avg       0.87      0.86      0.86      6193


Confusion Matrix: 
 [[3230  284]
 [ 553 2126]]

Accuracy : 0.8648474083642822


In [ ]:
#prediction and evaluation on test data with word2vec
pred_val = xgb_w2v.predict(X_val_w2vx) # Test Evaluation
print(classification_report(y_val_w2vx, pred_val))
print()
print('Confusion Matrix: \n', confusion_matrix(y_val_w2vx, pred_val))
print()
print('Accuracy :', accuracy_score(y_val_w2vx, pred_val))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       828
           1       0.78      0.68      0.73       592

    accuracy                           0.79      1420
   macro avg       0.79      0.77      0.78      1420
weighted avg       0.79      0.79      0.79      1420


Confusion Matrix: 
 [[717 111]
 [188 404]]

Accuracy : 0.7894366197183098


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Uyime's Project/Disaster Comparison Table for different Classifier.csv")
df

,Classifiers,Accuracy,Precision,recall,F1_Score
0,DT,0.68,0.73,0.71,0.72
1,XGB,0.79,0.79,0.87,0.83
2,SVM,0.81,0.81,0.90,0.85
3,BERT,0.82,0.00,0.00,0.00
4,RF,0.79,0.78,0.89,0.83
